In [1]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np

In [2]:
device_u = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device_v = torch.device('cuda:1' if torch.cuda.device_count() > 1 else device_u)
device_p = torch.device('cuda:2' if torch.cuda.device_count() > 2 else device_u)
device_T = torch.device('cuda:3' if torch.cuda.device_count() > 3 else device_u)

print(f"device_u: {device_u}")
print(f"device_v: {device_v}")
print(f"device_p: {device_p}")
print(f"device_T: {device_T}")

device_u: cuda:0
device_v: cuda:1
device_p: cuda:2
device_T: cuda:3


In [3]:
class FCN(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_layers=4, neurons=50):
        super(FCN, self).__init__()
        layers = [nn.Linear(in_dim, neurons), nn.Tanh()]
        for _ in range(hidden_layers - 1):
            layers += [nn.Linear(neurons, neurons), nn.Tanh()]
        layers.append(nn.Linear(neurons, out_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)
    

net_u = FCN(in_dim=3, out_dim=1).to(device_u)   # For u(x, y, t)
net_v = FCN(in_dim=3, out_dim=1).to(device_v)   # For v(x, y, t)
net_p = FCN(in_dim=3, out_dim=1).to(device_p)   # For p(x, y, t)
net_T = FCN(in_dim=3, out_dim=1).to(device_T)   # For T(x, y, t)

In [4]:
def generate_collocation_points(N):
    x = np.random.rand(N, 1)
    y = np.random.rand(N, 1)
    t = np.random.rand(N, 1)
    X = np.hstack((x, y, t))
    return torch.tensor(X, dtype=torch.float32, requires_grad=True)

N_f = 10000
X_f = generate_collocation_points(N_f)
X_f_u = X_f.clone().detach().to(device_u).requires_grad_(True)
X_f_v = X_f.clone().detach().to(device_v).requires_grad_(True)
X_f_p = X_f.clone().detach().to(device_p).requires_grad_(True)
X_f_T = X_f.clone().detach().to(device_T).requires_grad_(True)

In [16]:
def compute_residual_u():
    # Evaluate u and its derivatives on device_u
    u = net_u(X_f_u)
    grads_u = autograd.grad(u, X_f_u, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_t = grads_u[:, 2:3]
    u_x = grads_u[:, 0:1]
    u_y = grads_u[:, 1:2]
    u_xx = autograd.grad(u_x, X_f_u, grad_outputs=torch.ones_like(u_x), create_graph=True)[0][:, 0:1]
    u_yy = autograd.grad(u_y, X_f_u, grad_outputs=torch.ones_like(u_y), create_graph=True)[0][:, 1:2]

    # Get v and p as constants on device_u
    v = net_v(X_f_v).detach().to(device_u)
    p = net_p(X_f_p)#.to(device_u)
    # Spatial derivatives of pressure
    p_x = autograd.grad(p, X_f_p, grad_outputs=torch.ones_like(p), create_graph=True, allow_unused=True)[0][:, 0:1].to(device_u)

    nu = 0.01  # kinematic viscosity
    # Momentum-u residual
    res_u = u_t + u * u_x + v * u_y + p_x - nu * (u_xx + u_yy)
    return res_u

In [21]:
def compute_residual_v():
    # Evaluate v and its derivatives on device_v
    v = net_v(X_f_v)
    grads_v = autograd.grad(v, X_f_v, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_t = grads_v[:, 2:3]
    v_x = grads_v[:, 0:1]
    v_y = grads_v[:, 1:2]
    v_xx = autograd.grad(v_x, X_f_v, grad_outputs=torch.ones_like(v_x), create_graph=True)[0][:, 0:1]
    v_yy = autograd.grad(v_y, X_f_v, grad_outputs=torch.ones_like(v_y), create_graph=True)[0][:, 1:2]

    # Get u and p as constants on device_v
    u = net_u(X_f_u).detach().to(device_v)
    p = net_p(X_f_p)#.to(device_v)
    # Pressure derivatives
    p_y = autograd.grad(p, X_f_p, grad_outputs=torch.ones_like(p), create_graph=True)[0][:, 1:2].to(device_v)

    nu = 0.01
    # Momentum-v residual
    res_v = v_t + u * v_x + v * v_y + p_y - nu * (v_xx + v_yy)
    return res_v


In [7]:
def compute_residual_continuity():
    # Divergence of velocity field on GPU 0
    u = net_u(X_f_u)
    v = net_v(X_f_v)
    u_x = autograd.grad(u, X_f_u, grad_outputs=torch.ones_like(u), create_graph=True)[0][:, 0:1]
    v_y = autograd.grad(v, X_f_v, grad_outputs=torch.ones_like(v), create_graph=True)[0][:, 1:2]
    res_c = u_x + v_y.to(device_u)
    return res_c

In [8]:
def compute_residual_T():
    # Evaluate T and its derivatives on device_T
    T = net_T(X_f_T)
    grads_T = autograd.grad(T, X_f_T, grad_outputs=torch.ones_like(T), create_graph=True)[0]
    T_t = grads_T[:, 2:3]
    T_x = grads_T[:, 0:1]
    T_y = grads_T[:, 1:2]
    T_xx = autograd.grad(T_x, X_f_T, grad_outputs=torch.ones_like(T_x), create_graph=True)[0][:, 0:1]
    T_yy = autograd.grad(T_y, X_f_T, grad_outputs=torch.ones_like(T_y), create_graph=True)[0][:, 1:2]

    # Get u,v as constants on device_T
    u = net_u(X_f_u).detach().to(device_T)
    v = net_v(X_f_v).detach().to(device_T)

    alpha = 0.01  # thermal diffusivity
    # Energy equation residual
    res_T = T_t + u * T_x + v * T_y - alpha * (T_xx + T_yy)
    return res_T

In [9]:
def compute_losses():
    # Compute individual residuals
    res_u = compute_residual_u()
    res_v = compute_residual_v()
    res_c = compute_residual_continuity()
    res_T = compute_residual_T()
    # Aggregate into per-network losses (MSE) and return
    loss_u = torch.mean(res_u**2) + torch.mean(res_c**2)
    loss_v = torch.mean(res_v**2)
    loss_T = torch.mean(res_T**2)
    

    loss_v_u = loss_v.to(device_u)
    loss_T_u = loss_T.to(device_u)
    total_loss = (loss_u + loss_v_u + loss_T_u)


    return loss_u, loss_v, loss_T, total_loss

In [10]:
optimizer_u = torch.optim.Adam(net_u.parameters(), lr=1e-3)
optimizer_v = torch.optim.Adam(net_v.parameters(), lr=1e-3)
optimizer_T = torch.optim.Adam(net_T.parameters(), lr=1e-3)

In [22]:
num_epochs = 5000
for epoch in range(num_epochs):
    optimizer_u.zero_grad()
    optimizer_v.zero_grad()
    optimizer_T.zero_grad()

    loss_u, loss_v, loss_T, total_loss = compute_losses()

    loss_u.backward(retain_graph=True)
    loss_v.backward(retain_graph=True)
    loss_T.backward()

    optimizer_u.step()
    optimizer_v.step()
    optimizer_T.step()


    if epoch % 500 == 0:
        print(f"Epoch {epoch}: loss_u={loss_u.item():.3e}, loss_v={loss_v.item():.3e}, loss_T={loss_T.item():.3e}, total={total_loss.item():.3e}")

Epoch 0: loss_u=1.159e-02, loss_v=1.680e-03, loss_T=9.864e-04, total=1.425e-02
Epoch 500: loss_u=3.768e-06, loss_v=1.333e-06, loss_T=6.911e-09, total=5.108e-06
Epoch 1000: loss_u=1.428e-06, loss_v=3.448e-07, loss_T=3.462e-09, total=1.776e-06
Epoch 1500: loss_u=3.518e-07, loss_v=1.162e-07, loss_T=2.159e-09, total=4.701e-07
Epoch 2000: loss_u=2.152e-07, loss_v=1.145e-07, loss_T=1.255e-09, total=3.310e-07
Epoch 2500: loss_u=1.558e-07, loss_v=1.107e-07, loss_T=9.294e-10, total=2.674e-07
Epoch 3000: loss_u=2.498e-07, loss_v=1.288e-07, loss_T=8.930e-10, total=3.794e-07
Epoch 3500: loss_u=1.236e-07, loss_v=9.855e-08, loss_T=5.533e-10, total=2.227e-07
Epoch 4000: loss_u=2.294e-07, loss_v=1.049e-07, loss_T=3.881e-10, total=3.348e-07


KeyboardInterrupt: 

In [23]:
import matplotlib.pyplot as plt

